# ResNet

<div style="display: flex; align-items: center;">
    <img src="../imgs/ResNet.jpg" alt="Your Image" width="500" style="margin-right: 20px;">
    <div>
        <p>In chapter 7, we introduced a new dataset CIFAR, and tried to challenge it, but the result is limited by using the current knowledge we had learned. In the following chapters, we will aim to achieve a accuracy with 95% on both CIFAR10 and CIFAR100 and continuously introduce new models. In this chapter, we will introduce ResNet.</p>
        <p>From experience, the depth of the network is crucial to the performance of the model. When the number of network layers is increased, the network can extract more complex feature patterns, so theoretically, better results can be achieved when the model is deeper. But will deeper networks necessarily have better performance? As the depth of the network increases, the accuracy of the network saturates or even decreases.
</p>
        <p>The degradation problem of deep networks at least indicates that deep networks are not easy to train. Residual learning solves the problem of difficult training and even degradation of networks, making it easier to learn the original features directly compared to residual learning. When the residual is 0, the stacked layer only performs identity mapping, at least the network performance will not decrease. In fact, the residual will not be 0, which will enable the stacked layer to learn new features based on the input features, thus having better performance. This is somewhat similar to a "short circuit" in a circuit, so it is a short circuit connection.</p>
        <p></p>
    </div>
</div>

## ResNet

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet(nn.Module):
    def __init__(self, input_channels, output_size):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(input_channels, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.blocks_layer = nn.Sequential(
            self._make_layer(64),
            self._make_layer(128, stride=2),
            self._make_layer(256, stride=2)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, output_size)

    def _make_layer(self, out_channels, stride=1):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.blocks_layer(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(identity)
        out = self.relu(out)
        return out

# Train ResNet on CIFAR10/100

In [2]:
import sys
import torch.nn as nn
sys.path.append('../tools')
from CIFAR10 import CIFAR10Trainer
from CIFAR100 import CIFAR100Trainer

In [ ]:
model = ResNet(input_channels=3, output_size=10)
trainer = CIFAR10Trainer(model, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
trainer.train()
trainer.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-20 00:18:28
Epoch 1 / 30


[Valid]: 100%|██████████████████████| 40/40 [00:01<00:00, 31.92it/s, val_acc=59.2, val_loss=0.00917]


2024-05-20 00:18:42
Epoch 2 / 30


[Valid]: 100%|██████████████████████| 40/40 [00:01<00:00, 33.29it/s, val_acc=66.9, val_loss=0.00749]


2024-05-20 00:18:57
Epoch 3 / 30


...

[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 32.17it/s, val_acc=77.7, val_loss=0.0104]


2024-05-20 00:25:44
Epoch 29 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 31.05it/s, val_acc=77.9, val_loss=0.0107]


2024-05-20 00:26:00
Epoch 30 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.56it/s, val_acc=78.3, val_loss=0.0107]


In [ ]:
model_100 = ResNet(input_channels=3, output_size=100)
trainer_100 = CIFAR100Trainer(model_100, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
trainer_100.train()
trainer_100.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-20 00:27:53
Epoch 1 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.64it/s, val_acc=22.1, val_loss=0.0254]


2024-05-20 00:28:08
Epoch 2 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 32.79it/s, val_acc=31.8, val_loss=0.0212]


2024-05-20 00:28:23
Epoch 3 / 30


...

[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.44it/s, val_acc=46.4, val_loss=0.0239]


2024-05-20 00:35:06
Epoch 29 / 30


[Valid]: 100%|████████████████████████| 40/40 [00:01<00:00, 31.72it/s, val_acc=46.3, val_loss=0.024]


2024-05-20 00:35:21
Epoch 30 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.96it/s, val_acc=46.6, val_loss=0.0241]
